#An automated infrared spectral analysis using machine learning
By: Ray Gunawidjaja

Objectives
    1. Reduce the amount of data sets: 
        Perform analysis of data, cleaning, and EDA on NIST chemicals list
        
    2. Spectra acquisition and analysis:
        a. Download spectra from NIST database using the reduced NIST chemicals list
        b. Spectra analysis
        
    3. Modeling and testing:
        Select ML model, optimize, and test using challenging data sets    

Obj. 1 Reduce the amount of data sets:  

In [67]:
#Task 1. Funct: Read periodic table of elements and store into a dictionary: 
#    dict['element_name']=['atomic_number','atomic_mass'] 

import collections #create an ordered dictionary. In this case, order of elements is important.
dict = collections.OrderedDict()#initialize dictionary.

def dict_periodic_table(file_name):
    """ Read a text file line-by-line and extract information for: atomic number, atomic symbol, and relative atomic mass """
    
    file=open(file_name,'r') #open a file to read

    for line in file: #read file line-by-line, extract values, and store into a dictionary

        if line.startswith('Atomic Number'):
            line=line.replace(" ","")
            value1=line.split('=',1)

        if line.startswith('Atomic Symbol'):
            line=line.replace(" ","")
            key=line.split('=',1)

        if line.startswith('Relative Atomic Mass'):
            line=line.replace(" ","")
            value2=line.split('=',1)

            dict[key[1].strip()]=[value1[1].strip(),value2[1].strip()]
    
    return dict
####End of function

#1a. Call out function
file_name='NIST_periodic_table.txt'
dict=dict_periodic_table(file_name)

#1b.Test out dictionary by calling out an element
print(list(dict.keys()))

['H', 'D', 'T', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']


In [124]:
#Task2: Analyze NIST chemicals list containing 40K+ enteries and perform EDA with the aid of 
#    the above dict_periodic_table function  
    
import pandas as pd
import re

def read_and_clean_nist_chem_list(no_of_files=72619):
    """ Read and clean NIST chemicals list """ 
    
    #read file
    col_names=['Name', 'Formula', 'CAS']  
    #Rename columns 
    df=pd.read_csv("NIST_chemicals_list.csv", header=1, names=col_names)      
    #Remove rows that have null objects and keep only rows with no na values
    df=df.dropna()
    #make CAS the index column
    df=df.set_index('CAS')
    
    return df

def unique_elements(df,df_periodic_table):
    """ Extract unique elements and add to the list: unique_elements """
    
    #An array for storing element_list and unique_elements in NIST chemicals list
    elements_list=[]
    unique_elements=[]
    
    #create a list of elements from periodic table: elements_list
    for element in df_periodic_table:
        elements_list.append(element) 
        
    #determine distinct elements: check which element is present in formula. If not yet in unique_elements list, add it  
    for each in df.Formula:
        for element in elements_list:
            if (element in each) and (element not in unique_elements):
                unique_elements.append(element)
                
    return unique_elements

def filter_by_elements(df,elements_list):
    """ Filter compounds to contain any, but not exceeding the elements on the specified list: elements_list s"""
   
    #Create a blank pandas DataFrame
    new_df=pd.DataFrame()
    
    for i,element in enumerate(df.Formula):
        #strip cemical formula and leave out only the alphabets
        s_proc=re.sub('[^a-zA-Z]+', '', element)
        for allowed_element in elements_list:  
            #subtract stripped formula with elements list
            s_proc=s_proc.replace(allowed_element,"")
            #print("s_proc:", s_proc)
            #If the stripped formula is completely anihilated, then all elements must have matched
            #Thus, append compound to the new DataFrame
            if (s_proc==""):
            #    print("s_proc:", s_proc)
                new_df=new_df.append(df.iloc[i,:])
            #s_proc=re.findall('\D+', each) #extract only letters
            
    #relabel index column
    new_df.index.name="CAS"
    return new_df #returns a list

"""In progress
def calc_molec-weight(df):
    #Calculate molecular weight of a compound
    #This is a list for storing molecular weight of compounds
    mw_list=[]
    
    #separate alphabets from numerical
    for i,element in enumerate(df.Formula):
        #strip cemical formula and leave out only the alphabets
        #s_proc=re.sub('[^a-zA-Z-0-9]+', '', element)
        for allowed_element in elements_list:      
            s_proc=re.findall('\d+|\D+', df.Formula)
        
            mw_list.append()
"""

####End of functions
    
#1. call out function and inspect
df=read_and_clean_nist_chem_list()
#print(df.head()) #check

#2.determine unique elements in chemicals list
print("Unique elements:", unique_elements(df,list(dict.keys())))
print(len(unique_elements(df,list(dict.keys()))))

#3. filter compounds by elements
#Parse for alphabets in cemical formula and create a list.
#Compare list with master's list. They must be equal.

df_filtered=filter_by_elements(df.head(20),['C','H','O','N'])
print(df_filtered)
                                                 
#check                         
print("Unique elements:", unique_elements(filter_by_elements(df.head(20),['C','H','O','N']),list(dict.keys())))

#4. Calculate molecular weight of compounds
#In progress

Unique elements: ['H', 'C', 'N', 'O', 'Cl', 'Si', 'S', 'K', 'Sn', 'F', 'B', 'Br', 'Al', 'Nb', 'Mo', 'Ba', 'Ca', 'Nd', 'Se', 'D', 'Dy', 'Eu', 'In', 'I', 'P', 'Sb', 'T', 'Te', 'Th', 'U', 'Zr', 'Co', 'No', 'Ni', 'Fe', 'Hg', 'Pb', 'Cu', 'Mg', 'Ge', 'Tb', 'Rn', 'Cr', 'Ti', 'V', 'Re', 'La', 'Lu', 'Li', 'Tl', 'As', 'Cd', 'Ce', 'W', 'Er', 'Gd', 'Ho', 'Zn', 'Mn', 'Ga', 'Be', 'Au', 'Cs', 'Sr', 'Ag', 'Na', 'Pr', 'Sm', 'Y', 'Yb', 'Hf', 'Ru', 'Os', 'Pt', 'Rh', 'Ta', 'Ir', 'Pd', 'Bi', 'Rb', 'Sc', 'Pu', 'Tc', 'He', 'Ne', 'Xe', 'Np', 'Ar', 'Tm', 'Kr', 'Po', 'At', 'Es', 'Md', 'Pm', 'Pa', 'Ra', 'Ac', 'Am', 'Bk', 'Cm', 'Cf', 'Fm', 'Fr']
104
                Formula                                             Name
CAS                                                                     
100-01-6       C6H6N2O2                                   p-Nitroaniline
100-02-7        C6H5NO3                                 Phenol, 4-nitro-
100043-29-6       CH2N4                                     2H-Tetrazole
10004

In [122]:
#Obj. 2  Acquire reference spectra (in jcamp format) using the shortlist and analyze spectra

import urllib.request
import requests

def download_jcamp_from_nist(df_filtered,minimum_file_size=1000,no_of_files=20): 
        
    #NOTE THAT SOME OF THESE FILES TURN out to be empty.
    #Need to check file before proceeding. Don't download if empty. 
    #Files are saved in the same folder as this python script.
    
    for cas_no in df_filtered.index[:no_of_files]: #Download data based on CAS number partially
        #a first way
        url="http://webbook.nist.gov/cgi/cbook.cgi?JCAMP=C%s&Index=0&Type=IR" %(cas_no) #specify CAS no. separately
        
        if len(requests.get(url).content) >= 1000: #Download only if size is significant. 1000 seems optimum.
            #Strip comma, apostrophe, and space in the file name because it will mess up the ability of the jcamp program to read the file.
            url_ret=urllib.request.urlretrieve(url,df.loc[cas_no,'Name'].replace(",","").replace("'","").strip()+"_"+cas_no) #save file according to its cas_no
            print(url_ret) #print list of downloaded compounds
    
#Call out function to download
download_jcamp_from_nist(df_filtered,1000,20)  

('p-Nitroaniline_100-01-6', <http.client.HTTPMessage object at 0x000001598C71A160>)
('Phenol 4-nitro-_100-02-7', <http.client.HTTPMessage object at 0x000001598C71A400>)
('Cyclohexanol 2-(9h-purin-9-yl)- trans-_100057-61-2', <http.client.HTTPMessage object at 0x000001598C71A908>)
('9H-purine 9-cyclohex-2-enyl-6-hydrazino-_100058-19-3', <http.client.HTTPMessage object at 0x000001598C71A748>)
('Acetophenone 4-methoxy-_100-06-1', <http.client.HTTPMessage object at 0x000001598C71AE80>)
('Acetophenone 4-methoxy-_100-06-1', <http.client.HTTPMessage object at 0x000001598C71AB00>)
('(CH3)2C=CHC(CH3)=CH2_1000-86-8', <http.client.HTTPMessage object at 0x000001598C71A9B0>)
('(CH3)2C=CHC(CH3)=CH2_1000-86-8', <http.client.HTTPMessage object at 0x000001598C71A7F0>)
('(CH3)2C=CHC(CH3)=CH2_1000-86-8', <http.client.HTTPMessage object at 0x000001598C71A6A0>)
('23-Pentadiene 24-dimethyl-_1000-87-9', <http.client.HTTPMessage object at 0x000001598C71A7B8>)
('23-Pentadiene 24-dimethyl-_1000-87-9', <http.clie

In [123]:
#Obj. 3 Modeling and testing

#In progress